In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
import tensorflow as tf
from mpl_toolkits import mplot3d
from matplotlib import cm
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [ ]:
seed = 12
np.random.seed(seed)

In [ ]:
import pandas as pd
path = ('/content/fetal_healthClean.csv')
d1 = pd.read_csv(path)
d1.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,mean_value_of_long_term_variability,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0,0.000,0,0,0,73.0,0.5,43.0,2.4,62.0,126.0,2.0,0.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0,0.006,1,0,0,17.0,2.1,0.0,10.4,68.0,198.0,6.0,1.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0,0.008,1,0,0,16.0,2.1,0.0,13.4,68.0,198.0,5.0,1.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0,0.008,1,0,0,16.0,2.4,0.0,23.0,53.0,170.0,11.0,0.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0,0.008,0,0,0,16.0,2.4,0.0,19.9,53.0,170.0,9.0,0.0,138.0,11.0,1.0,1.0


In [ ]:
pd.to_numeric(d1['fetal_health'])

0       2.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2108    2.0
2109    2.0
2110    2.0
2111    2.0
2112    1.0
Name: fetal_health, Length: 2113, dtype: float64

In [ ]:
#split dataset in features and target variable
feature_cols = ['baseline value', 'accelerations', 'fetal_movement','uterine_contractions',
                'light_decelerations','severe_decelerations', 'prolongued_decelerations',
                'abnormal_short_term_variability', 'mean_value_of_short_term_variability',
                'percentage_of_time_with_abnormal_long_term_variability', 'mean_value_of_long_term_variability',
                'histogram_min', 'histogram_max', 'histogram_number_of_peaks', 'histogram_number_of_zeroes',
                'histogram_median', 'histogram_variance', 'histogram_tendency']
X = d1[feature_cols] # Features
y = d1.fetal_health # Target variable

In [ ]:
# Hyper-parameters
training_epochs = 400 # Total number of training epochs
learning_rate = 0.03 # The learning rate

In [ ]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
item_counts = d1['fetal_health'].value_counts(normalize=True)
item_counts

1.0    0.778987
2.0    0.138192
3.0    0.082821
Name: fetal_health, dtype: float64

In [ ]:
def create_model():
    model = Sequential()
    
    model.add(Dense(6, input_dim=18, kernel_initializer='normal', activation='relu'))
    
    
    model.add(Dense(y_train.T.shape[0], activation='sigmoid'))

    model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 114       
                                                                 
 dense_1 (Dense)             (None, 1690)              11830     
                                                                 
Total params: 11,944
Trainable params: 11,944
Non-trainable params: 0
_________________________________________________________________


In [ ]:
results = model.fit(
    X_train, y_train.T,
    epochs= training_epochs,
    batch_size = 516,
    validation_data = (X_test, y_test.T),
    verbose = 0
)

In [ ]:
prediction_values = model.predict(X_test)

In [ ]:
print("Test-Accuracy:","%.2f%%" % (np.mean(results.history["val_accuracy"])*100))


Test-Accuracy: 81.56%


In [ ]:
def create_model():
    model2 = Sequential()
    
    model2.add(Dense(54, input_dim=18, kernel_initializer='normal', activation='relu'))
    
    
    model2.add(Dense(y_train.T.shape[0], activation='sigmoid'))

    model2.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model
model2 = create_model()
model2.summary()

results2 = model2.fit(
    X_train, y_train.T,
    epochs= training_epochs,
    batch_size = 516,
    validation_data = (X_test, y_test.T),
    verbose = 0
)

prediction_values2 = model2.predict(X_test)
print("Test-Accuracy:","%.2f%%" % (np.mean(results2.history["val_accuracy"])*100))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 114       
                                                                 
 dense_1 (Dense)             (None, 1690)              11830     
                                                                 
Total params: 11,944
Trainable params: 11,944
Non-trainable params: 0
_________________________________________________________________
Test-Accuracy: 84.53%


In [ ]:
score = model2.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.42047372460365295
Test accuracy: 0.8368794322013855


In [ ]:
from sklearn.naive_bayes import GaussianNB
modelNB = GaussianNB()
modelNB.fit(X_train,y_train)
NBpredict = modelNB.predict(X_test)


In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, NBpredict))

Accuracy: 0.8652482269503546


In [ ]:
recallNB = metrics.recall_score(y_test, NBpredict, average ='macro')
print('Recall: %f' % recallNB)

Recall: 0.773208


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, NBpredict)

array([[290,  32,   5],
       [  7,  61,   0],
       [  2,  11,  15]])